In [4]:
import pandas as pd
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.metrics import smape, mae, rmse, mape
import tqdm as tqdm

In [9]:
df = pd.read_csv('ts_pred2.csv')

In [10]:
df.head()

,YearMonth,appointment_availability,online_systems,blood_tests,getting_through_on_phone,excellent_clinical_care,follow-up,noise,overall_excellent,prescriptions,reception_staff_friendly,reception_staff_rude,rushed_consultation,staff_kindness,staff_professionalism,telehealth,treatment_quality,vaccinations,waiting_time
0,2024-01-31,-0.342,-0.723,-0.730,-0.707,0.590,-0.845,0.145,0.816,-0.738,0.948,NaN,-0.525,0.712,0.728,-0.556,0.081,NaN,-0.731
1,2024-02-29,-0.373,-0.248,-0.903,-0.662,0.677,-0.287,-0.530,0.856,-0.676,0.798,-0.741,-0.724,0.747,0.143,-0.371,0.280,-0.282,-0.693
2,2024-03-31,-0.382,-0.453,-0.369,-0.751,0.822,-0.148,-0.372,0.898,-0.246,0.761,-0.441,-0.756,0.721,0.300,-0.260,0.311,0.054,-0.904
3,2024-04-30,-0.361,-0.424,-0.794,-0.779,0.633,-0.205,-0.496,0.885,-0.623,0.788,-0.694,-0.671,0.770,0.213,-0.313,0.220,-0.218,-0.670
4,2024-05-31,-0.258,-0.472,-0.747,-0.364,0.714,-0.287,-0.495,0.818,-0.285,0.744,-0.569,-0.470,0.543,0.189,-0.294,0.343,-0.643,-0.581


In [17]:
df_one = df[['YearMonth', 'online_systems']].copy()

In [18]:
df_one["YearMonth"] = pd.to_datetime(df_one["YearMonth"])
df_one = df_one.sort_values(by="YearMonth")

In [20]:
series = TimeSeries.from_dataframe(df_one,
                                   time_col="YearMonth",
                                   value_cols="online_systems",
                                   fill_missing_dates=True,
                                   freq=None)

In [21]:
series

,online_systems
YearMonth,
2024-01-31,-0.723
2024-02-29,-0.248
2024-03-31,-0.453
2024-04-30,-0.424
2024-05-31,-0.472
...,...
2025-08-31,-0.081
2025-09-30,-0.221
2025-10-31,-0.122


In [26]:
validation_length = 3
train_series, val_series = series[:-validation_length], series[-validation_length:]

,online_systems
YearMonth,
2025-10-31,-0.122
2025-11-30,0.134
2025-12-31,-0.377


In [28]:
input_chunk_length=6,
output_chunk_length=2,
n_epochs=40

pl_trainer_kwargs = {"accelerator": "cpu"}
optimizer_kwargs = {'lr': 1e-3}

model = NBEATSModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=output_chunk_length,
    output_chunk_shift=0,
    n_epochs=n_epochs,
    random_state=42,
    force_reset=True,
    save_checkpoints=False,
    pl_trainer_kwargs=pl_trainer_kwargs,
    optimizer_kwargs=optimizer_kwargs,
    # Default NBEATS generic architecture parameters
    num_stacks=30,
    num_blocks=1,
    num_layers=4,
    layer_widths=256,
    activation='ReLU',
    batch_size=32,
)
# Train the model on the scaled training data.
model.fit(train_series, verbose=True)


# 4. Make Predictions on Validation Set and Evaluate
print("Evaluating model on the validation set...")
val_predictions = model.predict(n=len(val_series))
# IMPORTANT: Inverse transform predictions to get them back to the original scale.
# val_predictions = scaler.inverse_transform(val_predictions_scaled)

# Initialize metrics to handle cases with no positive validation data
nbeats_smape, nbeats_mae, nbeats_rmse, nbeats_mape = [np.nan] * 4

# Calculate sMAPE on non-zero actuals to avoid division-by-zero errors.
# *** FIX: Convert to pandas Series to perform boolean filtering, as Darts TimeSeries doesn't support it. ***
val_series_pd = val_series.to_series()
val_series_non_zero_pd = val_series_pd[val_series_pd > 0]

if not val_series_non_zero_pd.empty:
    # Convert back to a TimeSeries for Darts metrics and functions
    val_series_non_zero = TimeSeries.from_series(val_series_non_zero_pd)
    val_predictions_filtered = val_predictions.slice_intersect(val_series_non_zero)

    # Calculate evaluation metrics
    nbeats_smape = smape(val_series_non_zero, val_predictions_filtered)
    nbeats_mae = mae(val_series_non_zero, val_predictions_filtered)
    nbeats_rmse = rmse(val_series_non_zero, val_predictions_filtered)
    nbeats_mape = mape(val_series_non_zero, val_predictions_filtered)

    print(f"-> NBEATS sMAPE on validation set (non-zero actuals): {nbeats_smape:.2f}%")
    print(f"-> NBEATS MAPE on validation set (non-zero actuals): {nbeats_mape:.2f}%")
    print(f"-> NBEATS MAE on validation set: {nbeats_mae:.2f}")
    print(f"-> NBEATS RMSE on validation set: {nbeats_rmse:.2f}")
else:
    print("-> No positive actuals in validation set for evaluation.")



TypeError: can only concatenate tuple (not "int") to tuple